# Build and Deploy an Agent Development Kit (ADK) Agent

> IMPORTANT: This is the third notebook in the lab. The notebooks  build on top of each other, so be sure to run the preceding notebooks, in order, before running this one. Start your journey building ADK Agents with MCP Toolbox [here](./1_setup_and_explore_databases.ipynb). 

## Overview

<img src="img/agent_development_kit_logo.png" align="left" style="height: 140px;"> Agent Development Kit (ADK) is a flexible and modular framework for developing and deploying AI agents. While optimized for Gemini and the Google ecosystem, ADK is model-agnostic, deployment-agnostic, and is built for compatibility with other frameworks. ADK was designed to make agent development feel more like software development, to make it easier for developers to create, deploy, and orchestrate agentic architectures that range from simple tasks to complex workflows.

In this notebook, you will build and deploy a sophisticated, enterprise-ready AI agent using the Agent Development Kit (ADK) and MCP Toolbox for Databases. You will start by creating and testing an agent that runs entirely on the local Vertex AI Workbench instance, and then transition it to a scalable, enterprise-ready architecture by deploying it as a secure microservice in Cloud Run. 

The key milestones you will achieve are:

- Building an Agent with MCP Toolbox: You will define and deploy an ADK agent that utilizes the MCP Toolbox to securely connect to the AlloyDB and Spanner databases provisioned in the previous labs. This will demonstrate how to create agents that can perform complex financial tasks, such as fraud detection, transaction lookups, and account analysis.
- Secure Deployment to Cloud Run: You will learn how to deploy the ADK agent to a secure Cloud Run environment within a private VPC. The deployment will emphasize enterprise security by using a dedicated service account, managing secrets with Secret Manager, and enabling Cloud Logging and Cloud Trace for robust observability.
- Enabling Telemetry and Observability: The notebook will guide you through enabling built-in tracing with ADK, allowing you to visualize the end-to-end execution of your agent's interactions, including tool calls made through the MCP Toolbox.
- Interacting with a Deployed Agent: You will learn how to programmatically invoke the deployed agent.

### Terraform Resources

The following pre-requisite resources were created for you by Terraform. See the [main.tf](../terraform/main.tf) file for more details on the environment configuration:

- Custom VPC (demo-vpc): Creates the demo-vpc private network where the Cloud Run services for the ADK agent and MCP Toolbox are deployed, ensuring secure communication between them.
- Cloud Router and Cloud NAT: Enables the Cloud Run services to access external resources securely without exposing them to the public internet.
- IAM Service Account: Defines the toolbox-service-account service account, which has the necessary permissions to access the AlloyDB and Spanner databases, as well as Secret Manager.
- Cloud SQL for PostgreSQL Instance: The Cloud SQL instance that is used by the ADK agent for session management, ensuring that conversation states are persisted securely.
- Cloud DNS: Sets up a private DNS zone to enable secure service-to-service communication between the ADK agent and MCP Toolbox within the VPC.

### Google Cloud Services Used

This notebook makes use of the following Google Cloud services:

- Cloud Run: Hosts the ADK agent and MCP Toolbox as secure, scalable services within a private VPC.
- Secret Manager: Securely stores the tools.yaml configuration and database credentials, which are accessed by the Cloud Run services at runtime.
- IAM (Identity and Access Management): Manages permissions for the service accounts used by the ADK agent and MCP Toolbox, ensuring least-privilege access to resources.
- Cloud SQL for PostgreSQL: Provides a persistent, managed database for the ADK agent's session management, enabling stateful conversations.
- AlloyDB for PostgreSQL: Serves as a high-performance data source for the MCP Toolbox, allowing the agent to query financial transaction data.
- Spanner: Acts as another critical data source, providing the agent with access to relational and graph data for complex queries.
- MCP Toolbox for Databases (Toolbox): An open-source MCP (Model Context Protocol) server that allows developers to connect gen AI agents to enterprise data easily and securely.
- Vertex AI Workbench: The development environment for running this notebook and interacting with the deployed services.
- Cloud Logging and Cloud Trace: Provide observability into the agent's operations, allowing you to monitor logs and trace requests as they flow through the system.

### Logical Flow

The notebook is structured to guide you through the complete lifecycle of building and deploying an ADK agent with MCP Toolbox tools:
- Basic Setup: This section prepares your environment by defining variables, connecting to your Google Cloud project, and installing the necessary Python libraries.
- Build an Agent with MCP Toolbox Tools: Here, you will define the ADK agent, specifying its model, name, description, and instructions. Crucially, you will load a toolset from the deployed MCP Toolbox, enabling the agent to interact with your databases.
- Manually Invoke the Agent: Before deploying, you will test the agent locally by sending it a series of prompts. This allows you to verify that the agent can correctly interpret the prompts and utilize the MCP Toolbox tools to retrieve information from the databases.
- Inspect Agent Processing: To gain deeper insight into the agent's behavior, you will enable detailed logging. This will reveal the step-by-step execution of the agent, including the specific tool calls it makes, which is invaluable for debugging and understanding the agent's logic.
- Deploy the ADK Agent to Cloud Run: This is a critical step where you will deploy the agent as a secure Cloud Run service. You will learn how to configure the service to use Cloud SQL for session persistence, ensuring that conversations are not lost upon server restarts. You will also enable Cloud Logging and Cloud Trace for observability.
- Invoke the Cloud Run Agent: Once deployed, you will interact with the agent running on Cloud Run. The notebook demonstrates how to handle authentication and session management when invoking a deployed agent, providing a template for building interactive applications.
- View Persisted ADK Data in Cloud SQL: To confirm that the session data is being persisted correctly, you will connect to the Cloud SQL instance and query the tables used by the ADK for session, event, and state management.
- View Telemetry: Finally, you will explore the traces generated by the agent's invocations in Cloud Trace. This will provide a visual representation of the request flow, from the user's prompt to the agent's response, including the tool calls made to the MCP Toolbox.

## Basic Setup

### Define Notebook Variables

Update the `project_id` and `region` variables below to match your environment. You can use defaults for the rest of the project variables. 

You will be prompted for the Cloud SQL password you chose then you provisioned the environment with Terraform. We'll use the Cloud SQL instance in this notebook to store ADK Session state. 

In [ ]:
# Project variables
project_id = "your-project"
region = "your-region"
vpc = "demo-vpc"
gcs_bucket_name = f"project-files-{project_id}"

# Cloud SQL Variables
cloud_sql_instance = "my-postgres-instance"
cloud_sql_database = "postgres"
cloud_sql_user = "postgres"
cloud_sql_password = input("Please enter the password for the Cloud SQL 'postgres' database user: ")

In [ ]:
# Set env variable to suppress annoying system warnings when running shell commands
%env GRPC_ENABLE_FORK_SUPPORT=1

### Connect to your Google Cloud Project

In [ ]:
# Configure gcloud.
!gcloud config set project {project_id}

### Configure Logging

In [ ]:
import logging
import sys

# Configure the root logger to output messages with ERROR level
logging.basicConfig(level=logging.ERROR, stream=sys.stdout, format='%(asctime)s[%(levelname)5s][%(name)14s] - %(message)s',  datefmt='%H:%M:%S', force=True)

### Install Dependencies

In [ ]:
! pip install --quiet toolbox-core==0.2.1 \
                      google-adk==1.5.0 \
                      google-genai==1.23.0 \
                      asyncpg==0.30.0 \
                      "cloud-sql-python-connector[asyncpg]"==1.18.2 \
                      opentelemetry-instrumentation-fastapi \
                      opentelemetry-exporter-gcp-trace


### Define Helper Functions

#### rest_api_helper()

In [ ]:
import requests
import google.auth
import json

# Get an access token based upon the current user
creds, _ = google.auth.default()
authed_session = google.auth.transport.requests.AuthorizedSession(creds)
access_token=creds.token

if project_id:
  authed_session.headers.update({"x-goog-user-project": project_id}) # Required to workaround a project quota bug

def rest_api_helper(
    url: str,
    http_verb: str,
    request_body: dict = None,
    params: dict = None,
    session: requests.Session = authed_session,
  ) -> dict:
  """Calls a REST API using a pre-authenticated requests Session."""

  headers = {"Content-Type": "application/json"}

  try:

    if http_verb == "GET":
      response = session.get(url, headers=headers, params=params)
    elif http_verb == "POST":
      response = session.post(url, json=request_body, headers=headers, params=params)
    elif http_verb == "PUT":
      response = session.put(url, json=request_body, headers=headers, params=params)
    elif http_verb == "PATCH":
      response = session.patch(url, json=request_body, headers=headers, params=params)
    elif http_verb == "DELETE":
      response = session.delete(url, headers=headers, params=params)
    else:
      raise ValueError(f"Unknown HTTP verb: {http_verb}")

    # Raise an exception for bad status codes (4xx or 5xx)
    response.raise_for_status()

    # Check if response has content before trying to parse JSON
    if response.content:
        return response.json()
    else:
        return {} # Return empty dict for empty responses (like 204 No Content)

  except requests.exceptions.RequestException as e:
      # Catch potential requests library errors (network, timeout, etc.)
      # Log detailed error information
      print(f"Request failed: {e}")
      if e.response is not None:
          print(f"Request URL: {e.request.url}")
          print(f"Request Headers: {e.request.headers}")
          print(f"Request Body: {e.request.body}")
          print(f"Response Status: {e.response.status_code}")
          print(f"Response Text: {e.response.text}")
          # Re-raise a more specific error or a custom one
          raise RuntimeError(f"API call failed with status {e.response.status_code}: {e.response.text}") from e
      else:
          raise RuntimeError(f"API call failed: {e}") from e
  except json.JSONDecodeError as e:
      print(f"Failed to decode JSON response: {e}")
      print(f"Response Text: {response.text}")
      raise RuntimeError(f"Invalid JSON received from API: {response.text}") from e


#### get_auth_token()

In [ ]:
import urllib

import google.auth.transport.requests
import google.oauth2.id_token


def get_auth_token(endpoint):
    # Cloud Run uses your service's hostname as the `audience` value
    # audience = 'https://my-cloud-run-service.run.app/'
    # For Cloud Run, `endpoint` is the URL (hostname + path) receiving the request
    # endpoint = 'https://my-cloud-run-service.run.app/my/awesome/url'
    
    auth_req = google.auth.transport.requests.Request()
    id_token = google.oauth2.id_token.fetch_id_token(auth_req, endpoint)

    return id_token

#### run_cloudsql_query()

In [ ]:
import sqlalchemy
from sqlalchemy import text, exc
import pandas as pd
import logging

async def run_cloudsql_query(pool, sql: str, params = None, output_as_df: bool = True):
    """Executes a SQL query or statement against the Cloud SQL database pool.

    Handles various SQL statements:
    - SELECT/WITH: Returns results as a DataFrame (if output_as_df=True)
      or ResultProxy. Supports parameters. Does not commit.
    - EXPLAIN/EXPLAIN ANALYZE: Executes the explain, returns the query plan
      as a formatted multi-line string. Ignores output_as_df.
      Supports parameters. Does not commit.
    - INSERT/UPDATE/DELETE/CREATE/ALTER etc.: Executes the statement,
      commits the transaction, logs info, and returns the ResultProxy.
      Supports single or bulk parameters (executemany).

    Args:
      pool: An asynchronous SQLAlchemy connection pool.
      sql: A string containing the SQL query or statement template.
      params: Optional.
        - None: Execute raw SQL (Use with caution for non-SELECT/EXPLAIN).
        - dict or tuple: Parameters for a single execution.
        - list of dicts/tuples: Parameters for bulk execution (executemany).
      output_as_df (bool): If True and query is SELECT/WITH, return pandas DataFrame.
                           Ignored for EXPLAIN and non-data-returning statements.

    Returns:
      pandas.DataFrame | str | sqlalchemy.engine.Result | None:
        - DataFrame: For SELECT/WITH if output_as_df=True.
        - str: For EXPLAIN/EXPLAIN ANALYZE, containing the formatted query plan.
        - ResultProxy: For non-SELECT/WITH/EXPLAIN statements, or SELECT/WITH
                       if output_as_df=False.
        - None: If a SQLAlchemy ProgrammingError or other specific error occurs.
    """
    sql_lower_stripped = sql.strip().lower()
    is_select_with = sql_lower_stripped.startswith(('select', 'with'))
    is_explain = sql_lower_stripped.startswith('explain')
    is_data_returning = is_select_with or is_explain
    effective_output_as_df = output_as_df and is_select_with
    is_bulk_operation = isinstance(params, list) and len(params) > 0

    async with pool.connect() as conn:
        try:
            if params:
                result = await conn.execute(text(sql), params)
            else:
                result = await conn.execute(text(sql))

            if is_data_returning:
                if is_explain:
                    try:
                        plan_rows = result.fetchall()
                        query_plan = "\n".join([str(row[0]) for row in plan_rows])
                        return query_plan
                    except Exception as e:
                        logging.error(f"Error fetching/formatting EXPLAIN result: {e}")
                        return None
                else:  # SELECT / WITH
                    if effective_output_as_df:
                        try:
                            rows = result.fetchall()
                            column_names = result.keys()
                            df = pd.DataFrame(rows, columns=column_names)
                            return df
                        except Exception as e:
                            logging.error(f"Error converting SELECT result to DataFrame: {e}")
                            return result
                    else:
                        return result
            else:
                await conn.commit()
                operation_type = sql.strip().split()[0].upper()
                row_count = result.rowcount

                if is_bulk_operation:
                    print(f"Bulk {operation_type} executed for {len(params)} items. Result rowcount: {row_count}")
                elif operation_type in ['INSERT', 'UPDATE', 'DELETE']:
                    print(f"{operation_type} statement executed successfully. {row_count} row(s) affected.")
                else:
                    print(f"{operation_type} statement executed successfully. Result rowcount: {row_count}")
                return result

        except exc.ProgrammingError as e:
            logging.error(f"SQL Programming Error executing query:\nSQL: {sql[:500]}...\nParams (sample): {str(params)[:500]}...\nError: {e}")
            return None
        except Exception as e:
            logging.error(f"An unexpected error occurred during query execution:\nSQL: {sql[:500]}...\nError: {e}")
            raise


## Build an Agent with MCP Toolbox Tools

### Get the Toolbox Endpoint

In [ ]:
import json

toolbox_url = ! gcloud run services describe toolbox --region {region} --format 'value(metadata.annotations."run.googleapis.com/urls")'
toolbox_url = json.loads(toolbox_url[0])[0]
print(toolbox_url)

### Define Agent

In [ ]:
# Reference: https://googleapis.github.io/genai-toolbox/getting-started/local_quickstart/

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService
from google import genai
from google.genai import types
from toolbox_core import ToolboxSyncClient

import asyncio
import os

# Use GOOGLE_API_KEY or Vertex AI
# os.environ['GOOGLE_API_KEY'] = 'your-api-key'

os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'TRUE'
os.environ['GOOGLE_CLOUD_PROJECT']      = project_id
os.environ['GOOGLE_CLOUD_LOCATION']     = region

# Get auth token to invoke Toolbox on Cloud Run
auth_token = get_auth_token(toolbox_url)

async def invoke_agent(query: str):
  with ToolboxSyncClient(
        toolbox_url,
        client_headers={"Authorization": f"Bearer {auth_token}"}
  ) as toolbox_client:

      prompt = """
        You're a helpful financial assistant. You handle fraud detection tasks, transaction lookups,
        account details, loan payments, suspicious activity reports, and other tasks related to 
        financial data. Always provide a customer_id when using tools to lookup information.
      """

      app_name = 'finance_agent'
    
      user_id = '123'

      root_agent = Agent(
          model='gemini-2.5-flash',
          name=app_name,
          description='A helpful AI assistant.',
          instruction=prompt,
          tools=toolbox_client.load_toolset("finance-toolset"),
      )

      session_service = InMemorySessionService()
      artifacts_service = InMemoryArtifactService()
      session = await session_service.create_session(
          state={}, app_name=app_name, user_id=user_id
      )
      runner = Runner(
          app_name=app_name,
          agent=root_agent,
          artifact_service=artifacts_service,
          session_service=session_service,
      )

      content = types.Content(role='user', parts=[types.Part(text=query)])
      events = runner.run(session_id=session.id,
                          user_id=user_id, new_message=content)

      responses = (
        part.text
        for event in events
        for part in event.content.parts
        if part.text is not None
      )

      for text in responses:
        print(text)


### Manually Invoke the Agent

In [ ]:
prompts = [
    "What are the latest trasactions for customer_id 11?",
    "Tell me about the recent account transfers made by customer_id 8 and flag any suspicious activity.",
    "Is anything strange about the latest tranactions made by customer_id 3?",
]

for p in prompts:
    print(f"User prompt: {p}\n")
    print("Agent response:")
    response = await invoke_agent(p)
    print("\n\n")
    

## Inspect Agent Processing

The examples above show that the Agent has access to our databases via MCP Toolbox, because the output contains data the Agent wouldn't otherwise know. But what if we want to debug an Agent's logic or understand the specific tool calls that were made? When developing locally, we can set the log level to info to see detailed, step-by-step Agent execution and tool call details. 

### Set log level to INFO

In [ ]:
import logging
import sys

# Configure the root logger to output messages with INFO level or above
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(asctime)s[%(levelname)5s][%(name)14s] - %(message)s',  datefmt='%H:%M:%S', force=True)

### Examine Detailed Output

In [ ]:
prompt = "How many transfers has customer_id 462 made recently? Is there anything strange about the transfers?"
await invoke_agent(prompt)

## Deploy the ADK Agent to Cloud Run

Now we will deploy the ADK agent to Cloud Run. The deployment process will use a `Dockerfile` to containerize the application and configure it to connect to the Cloud SQL instance for session management. We will also enable Cloud Logging and Cloud Trace for observability.

References:
- https://github.com/alphinside/deploy-and-manage-adk-service/tree/main/weather_agent
- https://codelabs.developers.google.com/deploy-manage-observe-adk-cloud-run#1

### Use Cloud SQL for Session Persistence

The Agent Development Kit (ADK) uses a Session Service to maintain the state of conversations. By default, it uses an in-memory session service, but this is not suitable for production environments where the server may restart, and sessions would be lost. To ensure persistence, we can connect the ADK to a Cloud SQL database to act as the persistent store for the Session service.

In [ ]:
# Get Cloud SQL connection name
cloud_sql_psc_ip = !gcloud compute forwarding-rules describe psc-forwarding-rule-{cloud_sql_instance} \
    --region={region} \
    --project={project_id} \
    --format="value(IPAddress)"
cloud_sql_psc_ip = cloud_sql_psc_ip[0]

# Construct the SESSION_SERVICE_URI
session_service_uri = f"postgresql+pg8000://postgres:{cloud_sql_password}@{cloud_sql_psc_ip}:5432/postgres"

print(f"SESSION_SERVICE_URI: {session_service_uri}")


### Create `__init__.py`

In [ ]:
# Initialize the agent directory
! mkdir -p deploy
! mkdir -p deploy/finance_agent

In [ ]:
%%writefile deploy/finance_agent/__init__.py
# __init__.py

from .agent import root_agent

__all__ = ["root_agent"]

### Create `agent.py`

In [ ]:
%%writefile deploy/finance_agent/agent.py
import os
import google.auth
import google.auth.transport.requests
import google.oauth2.id_token
from google.adk.agents import Agent
from toolbox_core import ToolboxSyncClient
from opentelemetry.instrumentation.aiohttp_client import (AioHttpClientInstrumentor)

# --- Environment and Authentication ---

# It's best practice to set these in the Cloud Run service's environment variables
# instead of hardcoding them in the script.
# os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'TRUE'
# os.environ['GOOGLE_CLOUD_PROJECT']      = 'your-project-id'
# os.environ['GOOGLE_CLOUD_LOCATION']     = 'your-region'

def get_auth_token(endpoint: str) -> str:
    """Fetches an OIDC token for authenticating with a Cloud Run service."""
    auth_req = google.auth.transport.requests.Request()
    return google.oauth2.id_token.fetch_id_token(auth_req, endpoint)

# The URL for the toolbox service should be passed as an environment variable
# for better flexibility between different environments (dev, prod).
TOOLBOX_URL = os.getenv("TOOLBOX_URL")
if not TOOLBOX_URL:
    raise ValueError("TOOLBOX_URL environment variable not set.")

# Get an auth token to invoke the Toolbox on Cloud Run
auth_token = get_auth_token(TOOLBOX_URL)

# --- Agent Definition ---

# Initialize the Toolbox client once and reuse it
print(f"Connecting to toolbox server at {TOOLBOX_URL}")
toolbox_client = ToolboxSyncClient(
    TOOLBOX_URL,
    client_headers={"Authorization": f"Bearer {auth_token}"}
)

# Define the agent's instructions
prompt = """
You're a helpful financial assistant. You handle fraud detection tasks, transaction lookups,
account details, loan payments, suspicious activity reports, and other tasks related to
financial data. Always provide a customer_id when using tools to lookup information.
"""

# Define the root agent
root_agent = Agent(
    model='gemini-2.5-flash',
    name='finance_agent',
    description='A helpful AI assistant for financial tasks.',
    instruction=prompt,
    # Load the tools from your MCP Toolbox
    tools=toolbox_client.load_toolset("finance-toolset"),
)

# Enable the AioHttp library for Toolbox tracing
AioHttpClientInstrumentor().instrument()

### Create `server.py`

In [ ]:
%%writefile deploy/server.py
import os
from fastapi import FastAPI
from google.adk.cli.fast_api import get_fast_api_app
from google.cloud import logging as google_cloud_logging

# OTEL imports
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider, export
from opentelemetry.propagate import set_global_textmap
from opentelemetry.propagators.cloud_trace_propagator import CloudTraceFormatPropagator
from opentelemetry.exporter.cloud_trace import CloudTraceSpanExporter

# --- Server Setup ---

# Initialize the Google Cloud Logging client
logging_client = google_cloud_logging.Client()
logger = logging_client.logger(__name__)

# The directory where your agent.py and __init__.py are located
AGENT_DIR = os.path.dirname(os.path.abspath(__file__))

# Get the session service URI from environment variables.
# This is where you'll provide the connection string for Cloud SQL.
session_uri = os.getenv("SESSION_SERVICE_URI")

# Prepare arguments for the FastAPI app
app_args = {"agents_dir": AGENT_DIR, "web": True}

# Use the Cloud SQL session service if the URI is provided
if session_uri:
    app_args["session_service_uri"] = session_uri
    logger.log_text(f"Using database session service.", severity="INFO")
else:
    logger.log_text(
        "SESSION_SERVICE_URI not provided. Using in-memory session service. "
        "Sessions will be lost on server restart.",
        severity="WARNING",
    )
    
# Observability
provider = TracerProvider()
processor = export.BatchSpanProcessor(CloudTraceSpanExporter())
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)
print("Set trace provider")

# Create the FastAPI app using the ADK utility
app: FastAPI = get_fast_api_app(**app_args)

app.title = "finance-agent"
app.description = "API for interacting with the ADK Finance Agent"

# Main execution
if __name__ == "__main__":
    import uvicorn
    
    # Enable the AioHttp library for Toolbox tracing
    AioHttpClientInstrumentor().instrument()

    # Instrument the FastAPI app.
    # This automatically creates parent spans for requests and propagates the trace context.
    FastAPIInstrumentor.instrument_app(app)

    uvicorn.run(app, host="0.0.0.0", port=8080)
    
    

### Create `pyproject.toml`

In [ ]:
%%writefile deploy/pyproject.toml
[project]
name = "deploy-and-manage-adk-service"
version = "0.1.0"
description = "Add your description here"
readme = "README.md"
requires-python = ">=3.12"
dependencies = [
    "google-adk==1.5.0",
    "locust==2.37.10",
    "pg8000==1.31.2",
    "python-dotenv==1.1.0",
    "toolbox-core==0.2.1",
    "google-genai==1.23.0",
    "opentelemetry-instrumentation-fastapi==0.55b1",
    "opentelemetry-api>=1.20.0",
    "opentelemetry-sdk>=1.20.0",
    "opentelemetry-instrumentation-fastapi>=0.40.0",
    "opentelemetry-exporter-gcp-trace>=1.9.0",
    "opentelemetry-propagator-gcp==1.9.0",
    "opentelemetry-exporter-gcp-monitoring==1.9.0a0",
    "opentelemetry-exporter-otlp",
    "opentelemetry-instrumentation-requests==0.55b1",
    "opentelemetry-instrumentation-aiohttp-client"
]

[dependency-groups]
dev = [
    "pytest==8.4.0",
    "ruff==0.11.13",
]

### Create a Dockerfile

In [ ]:
%%writefile deploy/Dockerfile
FROM python:3.12-slim
RUN pip install --no-cache-dir uv==0.7.13
WORKDIR /app
COPY . .
RUN uv sync
EXPOSE 8080
CMD ["uv", "run", "uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8080"]


### Deploy the Agent to Cloud Run

Here, we deploy the Finance Agent as a secure Cloud Run service. 

Key security configurations:
- `--network={vpc}` / `--subnet={vpc}`: Deploys the service within our private VPC.
- `--service-account=adk-service-account`: Uses a dedicated service account with least-privilege permissions.
- `--no-allow-unauthenticated`: Enforces IAM authentication for all invocations.
- `--ingress=internal`: Restricts network traffic to internal sources only.

In [ ]:
!gcloud run deploy finance-agent --quiet --no-user-output-enabled \
    --source ./deploy/ \
    --network={vpc} \
    --subnet={vpc} \
    --ingress=internal \
    --port 8080 \
    --project {project_id} \
    --region {region} \
    --no-allow-unauthenticated \
    --update-env-vars=TOOLBOX_URL={toolbox_url},GOOGLE_GENAI_USE_VERTEXAI=True,SESSION_SERVICE_URI={session_service_uri},GOOGLE_CLOUD_PROJECT={project_id},GOOGLE_CLOUD_LOCATION={region},GCS_BUCKET_NAME={gcs_bucket_name} \
    --min=1 \
    --service-account=adk-service-account \
    --memory=1Gi
    

### Grant finance-agent Invoke on toolbox

In [ ]:
# Get account finance-agent service account
service_account = !gcloud run services describe finance-agent \
    --region={region} \
    --format="value(spec.template.spec.serviceAccountName)"
service_account = service_account[0]
service_account = f"{service_account}@{project_id}.iam.gserviceaccount.com"

In [ ]:
# Grant service account invoke permissions on toolbox
!gcloud run services add-iam-policy-binding toolbox --region={region} \
    --member=serviceAccount:{service_account} \
    --role="roles/run.invoker"

### Enable Inter-service Private Network Connectivity

By default, Cloud Run endpoints are public. Since we've locked down the Cloud Run instances to only accept traffic from internal sources, we need to create DNS records to resolve Cloud Run endpoints to private IPs and enable Google Private Access in the `demo-vpc` subnet to enable secure service-to-service communication.

In [ ]:
!gcloud dns managed-zones create private-run-app-zone \
    --description="Private DNS zone for internal Cloud Run traffic" \
    --dns-name="run.app." \
    --visibility=private \
    --networks={vpc}

# Add the 'A' record
!gcloud dns record-sets create run.app. --zone=private-run-app-zone \
    --type=A --ttl=300 \
    --rrdatas="199.36.153.4,199.36.153.5,199.36.153.6,199.36.153.7"

# Add the CNAME record for wildcards
!gcloud dns record-sets create "*.run.app." --zone=private-run-app-zone \
    --type=CNAME --ttl=300 \
    --rrdatas="run.app."

# Enable Private Google Access on the subnet
!gcloud compute networks subnets update {vpc} \
    --region={region} \
    --enable-private-ip-google-access

## Invoke the Cloud Run Agent

### Define Functions for Agent Invocation

In [ ]:
import requests
import json
import uuid
import google.auth
import google.auth.transport.requests
import google.oauth2.id_token

# --- Session Tracking ---
# Use a set to keep track of sessions created during this notebook's execution.
created_sessions = set()

def create_agent_session(agent_url: str, auth_token: str, app_name: str, user_id: str, session_id: str) -> bool:
    """
    Explicitly creates a new session for the agent.
    Returns True if successful, False otherwise.
    """
    session_url = f"{agent_url}/apps/{app_name}/users/{user_id}/sessions/{session_id}"
    headers = {
        "Authorization": f"Bearer {auth_token}",
        "Content-Type": "application/json"
    }
    payload = {"state": {}}

    try:
        response = requests.post(session_url, headers=headers, json=payload, timeout=60)
        response.raise_for_status()
        print(f"Successfully created session: {session_id}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"\nError creating session: {e}")
        if e.response is not None:
            print(f"Response Status: {e.response.status_code}")
            print(f"Response Text: {e.response.text}")
        return False


def invoke_agent_streaming(query: str, session_id: str = None, user_id: str = 'default-user') -> None:
    """
    Invokes the agent on Cloud Run, creating a session only if it's new.
    """
    if session_id is None:
        session_id = str(uuid.uuid4())
    
    app_name = "finance_agent"

    try:
        agent_url = !gcloud run services describe finance-agent --region {region} --format 'value(status.url)'
        agent_url = agent_url[0]
        agent_auth_token = get_auth_token(agent_url)

        # --- Step 1: Create Session ONLY IF IT'S NEW ---
        if session_id not in created_sessions:
            if create_agent_session(agent_url, agent_auth_token, app_name, user_id, session_id):
                # Add the new session_id to our set of tracked sessions
                created_sessions.add(session_id)
            else:
                return # Stop if session creation fails

        # --- Step 2: Send the Message ---
        invoke_url = f"{agent_url}/run_sse"
        headers = {
            "Accept": "text/event-stream",
            "Authorization": f"Bearer {agent_auth_token}",
            "Content-Type": "application/json"
        }
        payload = {
            "app_name": app_name,
            "session_id": session_id,
            "user_id": user_id,
            "new_message": {
                "role": "user",
                "parts": [{"text": query}]
            }
        }

        with requests.post(invoke_url, headers=headers, json=payload, stream=True, timeout=300) as response:
            response.raise_for_status()
            print(f"\nUser Query: {query}")
            print("Agent Response:")
            for line in response.iter_lines():
                if line and line.decode('utf-8').startswith('data: '):
                    try:
                        json_data = json.loads(line.decode('utf-8')[6:])
                        text = json_data.get('content', {}).get('parts', [{}])[0].get('text', '')
                        print(text, end='', flush=True)
                    except (json.JSONDecodeError, IndexError):
                        continue
            print("\n" + "="*50 + "\n")

    except requests.exceptions.RequestException as e:
        print(f"\nError invoking agent: {e}")
        if e.response is not None:
            print(f"Response Status: {e.response.status_code}")
            print(f"Response Text: {e.response.text}")




### Query the Agent

We'll iterate over a few three-turn conversations with different customer ID's to show the agent working and to generate some traffic to be sampled by Cloud Trace. 

In [ ]:
customer_array = [462, 1, 32]

for c in customer_array:

    # A unique ID for the entire conversation
    conversation_session_id = f"session_{uuid.uuid4()}"
    conversation_user_id = f"user_{uuid.uuid4()}"

    # The first call will see the session_id is new, create it, and add it to the 'created_sessions' set.
    invoke_agent_streaming(
        f"Tell me about recent transfers and transactions by customer_id {c}.",
        session_id=conversation_session_id,
        user_id=conversation_user_id
    )

    # The second and subsequent calls will find the session_id in the set and skip the creation step.
    invoke_agent_streaming(
        "I'm a fraud analyst investigating these transactions and transfers. Does anything suspicious stand out about them?",
        session_id=conversation_session_id,
        user_id=conversation_user_id
    )

    invoke_agent_streaming(
        "What are some potential next steps for the investigation?",
        session_id=conversation_session_id,
        user_id=conversation_user_id
    )

## View Persisted ADK Data in Cloud SQL

### Connect to Cloud SQL

In [ ]:
import asyncpg
from google.cloud.sql.connector import Connector as CloudSqlConnector
import sqlalchemy
from sqlalchemy.ext.asyncio import create_async_engine

# Cloud SQL instance connection string
instance_connection_string = f"{project_id}:{region}:{cloud_sql_instance}"


async def init_connection_pool(connector: CloudSqlConnector, pool_size: int = 5) -> sqlalchemy.ext.asyncio.AsyncEngine:
    """
    Initializes an asynchronous SQLAlchemy connection pool for a Cloud SQL instance.
    """
    async def getconn() -> asyncpg.Connection:
        conn: asyncpg.Connection = await connector.connect_async(
            instance_connection_string,
            "asyncpg",
            user=cloud_sql_user,
            password=cloud_sql_password,
            db=cloud_sql_database,
            ip_type="psc"
        )
        return conn

    pool = create_async_engine(
        "postgresql+asyncpg://",
        async_creator=getconn,
        pool_size=pool_size,
        max_overflow=2,
        pool_timeout=30,
        pool_recycle=1800,
    )
    return pool


# Initialize the Cloud SQL Connector
connector = CloudSqlConnector()

# Initialize the connection pool
cloud_sql_pool = await init_connection_pool(connector)

### View Session Data

When a user starts interacting with your agent, the SessionService creates a Session object (google.adk.sessions.Session). This object acts as the container holding everything related to that one specific chat thread. Read more about this data [in the docs](https://google.github.io/adk-docs/sessions/session/)

In [ ]:
sql = "SELECT * FROM sessions LIMIT 10;"
await run_cloudsql_query(cloud_sql_pool, sql)

### View Event Data

Events are the fundamental units of information flow within the Agent Development Kit (ADK). They represent every significant occurrence during an agent's interaction lifecycle, from initial user input to the final response and all the steps in between. Read more about this data [in the docs](https://google.github.io/adk-docs/events/).

In [ ]:
sql = "SELECT * FROM events LIMIT 10;"
await run_cloudsql_query(cloud_sql_pool, sql)

### State Data

Within each Session (our conversation thread), the state attribute acts like the agent's dedicated scratchpad for that specific interaction. While session.events holds the full history, session.state is where the agent stores and updates dynamic details needed during the conversation. Read more about this data [in the docs](https://google.github.io/adk-docs/sessions/state/).

#### View User State Data

In [ ]:
sql = "SELECT * FROM user_states LIMIT 10;"
await run_cloudsql_query(cloud_sql_pool, sql)

#### View App State Data

In [ ]:
sql = "SELECT * FROM app_states LIMIT 10;"
await run_cloudsql_query(cloud_sql_pool, sql)

## View Telemetry

ADK supports built-in tracing. Navigate to the Cloud Trace to view traces for the Agent invocations above:

Select one of the traces, and you will see an end-to-end visualization of the Agent invocation, including MCP Toolbox tool calls as shown in the example below.

![Cloud Trace](img/cloud_trace.png)

Congratulations, you have completed Module 3! Proceed to [`4_configure_alloydb_natural_language.ipynb`](./4_configure_alloydb_natural_language.ipynb) to configure the powerful and flexible Natural Language feature of AlloyDB AI.